## Timing

In [5]:
import time

In [17]:
# Example 1
def logtime(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time
        with open('timelog.txt', 'a') as outfile:
            outfile.write(f'{time.ctime()}\t{func.__name__}\t{total_time}\n')
        return result
    return wrapper

In [18]:
@logtime
def slow_add(a, b):
    time.sleep(2)
    return a + b

In [19]:
@logtime
def slow_mul(a, b):
    time.sleep(3)
    return a * b

In [20]:
slow_add(5, 6)

11

In [21]:
slow_mul(5, 6)

30

In [16]:
time.ctime()

'Tue May  7 20:24:26 2019'

## Run Once per minute
Raise an exception if we try to run a function more than once every 60 seconds

In [27]:
def once_per_minute(func):
    last_invoked = 0
    def wrapper(*args, **kwargs):
        nonlocal last_invoked
        elapsed_time = time.time() - last_invoked
        if elapsed_time < 60:
            raise CalledTooOftenError(f"Only {elapsed_time} has passed")
        last_invoked = time.time()
        return func(*args, **kwargs)
    return wrapper

In [28]:
@once_per_minute
def add(a, b):
    return a + b

In [29]:
print(add(2, 3))

5


In [30]:
print(add(3, 4))

NameError: name 'CalledTooOftenError' is not defined

## Once per n
Raise an error if we try to run a function more than once in n seconds

In [31]:
def once_per_n(n):
    def middle(func):
        last_invoked = 0
        def wrapper(*args, **kwargs):
            nonlocal last_invoked
            if time.time() - last_invoked < n:
                raise CalledTooOftenError(f"Only {elapsed_time} has passed")
            last_invoked = time.time()
            return func(*args, **kwargs)
        return wrapper
    return middle

In [32]:
@once_per_n(20)
def add_van(a, b):
    return a + b

In [33]:
print(add_van(3, 3))

6


In [34]:
print(add_van(4,4))

NameError: name 'CalledTooOftenError' is not defined

## Example 4: Memoization
Cache the results of function calls, so we don't need to call them again

In [35]:
def memoize(func):
    cache = {}
    def wrapper(*args, **kwargs):
        if args not in cache:
            print(f"Caching NEW value for {func.__name__}{args}")
            cache[args] = func(*args, **kwargs)
        else:
            print(f"Using OLD value for {func.__name__}{args}")
        return cache[args]
    return wrapper

In [36]:
@memoize
def add_new(a, b):
    print("Running add_new!")
    return a + b

In [37]:
@memoize
def mul_new(a, b):
    print("Running mul_new!")
    return a * b

In [38]:
print(add_new(4, 5))
print(mul_new(3, 4))

Caching NEW value for add_new(4, 5)
Running Add!
9
Caching NEW value for mul_new(3, 4)
Running mul_new!
12


In [39]:
print(add_new(4, 5))
print(mul_new(3, 4))

Using OLD value for add_new(4, 5)
9
Using OLD value for mul_new(3, 4)
12


In [40]:
import pickle

In [41]:
def memoize2(func):
    cache = {}
    def wrapper(*args, **kwargs):
        t = (pickle.dumps(args), pickle.dumps(kwargs))
        if t not in cache:
            print(f"Caching NEW value for {func.__name__}{args}")
            cache[t] = func(*args, **kwargs)
        else:
            print(f"Using OLD value for {func.__name__}{args}")
        return cache[t]
    return wrapper

## Example 5: Attributes
Give many objects the same attributes, but without using inheritance

In [60]:
def fancy_repr(self):
#     return f"I'm a {type(self)}, with vars {vars(self)}"
    return f"I'm a {self.__class__.__name__}, with vars {vars(self)}"

In [61]:
def better_repr(c):
    c.__repr__ = fancy_repr
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        return o
    return wrapper

In [62]:
def better_repr(c):
    c.__repr__ = fancy_repr
    return c

In [63]:
@better_repr
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [64]:
f = Foo(10, [10, 20, 30])
print(f)

I'm a Foo, with vars {'x': 10, 'y': [10, 20, 30]}


In [65]:
f

I'm a Foo, with vars {'x': 10, 'y': [10, 20, 30]}

In [66]:
type(f)

__main__.Foo

In [67]:
Foo.__name__

'Foo'

In [68]:
vars(f)

{'x': 10, 'y': [10, 20, 30]}

In [70]:
def object_birthday(c):
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper

In [71]:
@object_birthday
class Boo():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [72]:
b = Boo(12, [12,22,32])
print(b)

In [74]:
print(b._created_at)

1557285474.3280833


In [75]:
def object_birthday(c):
    c.__repr__ = fancy_repr
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper

In [76]:
@object_birthday
class Noo():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [77]:
n = Noo(15, [15, 25, 35])
print(n)

I'm a Noo, with vars {'x': 15, 'y': [15, 25, 35], '_created_at': 1557285770.6747556}
